In [6]:
import json
import pandas as pd
from transformers import pipeline

In [77]:
from tqdm import tqdm, trange

In [13]:
data = json.load(open('data/example_data.json'))

In [28]:
pipe = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [66]:
pipe_sum = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6")

Error while downloading from https://cdn-lfs.hf.co/sshleifer/distilbart-cnn-6-6/68f994703e838fff13b9e54ab72d5ae44f184cb55300a7f65f46a7d282e8da23?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1731439452&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTQzOTQ1Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zc2hsZWlmZXIvZGlzdGlsYmFydC1jbm4tNi02LzY4Zjk5NDcwM2U4MzhmZmYxM2I5ZTU0YWI3MmQ1YWU0NGYxODRjYjU1MzAwYTdmNjVmNDZhN2QyODJlOGRhMjM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=MgriNufzU%7EVINKvHUaTmTwQDz80voHXdMtc7RY4aLs8eg0wWLztFiYIXrWVgyJ3LRDeX7FWjO9bVurKHAuFaqZl6Ukb3E9cEGA67T5H8UnO3DqlGya3gRwMU5eSSrpPRBfiVOSs9hvQ6m3Xv6pHm5-VxW5HzonayaVZoVvi-qc2DWyWtiQIIgZvea2L1Yois45Oon3TvbxJbB2ZuH6LCmwgpeFgfs7mosriMoHuksvrPm8l73bNE%7E1JlOKvaGpV6IgQgohAPC2ztvZI1esEIdVLDqwK8EPuZntqGU8

In [67]:
text = """You can get to CTM in different ways and yes, it's definitely possible to do so after some experience in start-up. However, I read that the most common route would be by being a CRA first instead. Something like CRA-Sr. CRA-CTM. 
If you start from study start-up you might have to spend some time as a regulatory or contract specialist before moving on as a Project Manager and then apply for CTM roles. Mind you though, this is what I see from people joining and leaving the company I work for.

The only reference point I have regarding salaries, is for a CRA position back in 2022; 38k base + 5k car allowance. That was outside of London. You'll definitely find more in the pinned post in this sub.
I was out of university and unemployed, was sending my CV to so many jobs both research and non-research, even applied for some mcjobs and spending hours tailoring my CV and spending lots of time making cover letters for those jobs. 
"""

text = """Language models are increasingly being deployed for general problem solving
across a wide range of tasks, but are still confined to token-level, left-to-right
decision-making processes during inference. This means they can fall short in
tasks that require exploration, strategic lookahead, or where initial decisions play
a pivotal role. To surmount these challenges, we introduce a new framework for
language model inference, “Tree of Thoughts” (ToT), which generalizes over the
popular “Chain of Thought” approach to prompting language models, and enables
exploration over coherent units of text (“thoughts”) that serve as intermediate steps
toward problem solving. ToT allows LMs to perform deliberate decision making
by considering multiple different reasoning paths and self-evaluating choices to
decide the next course of action, as well as looking ahead or backtracking when
necessary to make global choices. Our experiments show that ToT significantly
enhances language models problem-solving abilities on three novel tasks requiring
non-trivial planning or search: Game of 24, Creative Writing, and Mini Crosswords.
For instance, in Game of 24, while GPT-4 with chain-of-thought prompting only
solved 4%of tasks, our method achieved a success rate of 74%. Code repo with all
prompts: https://github.com/princeton-nlp/tree-of-thought-llm.


"""

In [70]:
res = pipe_sum("In 100 words, Can you summarize the following text\n " + text)

In [64]:
res

[{'summary_text': ' In our series of letters from the University of New York.'}]

In [29]:
data.keys()

dict_keys(['clinicalresearch+clinical trial', 'clinicalresearch+research study', 'clinicalresearch+new treatment', 'clinicalresearch+volunteer', 'health+clinical trial', 'health+research study', 'health+new treatment', 'health+volunteer', 'depression+clinical trial', 'depression+research study', 'depression+new treatment', 'depression+volunteer', 'mentalhealth+clinical trial', 'mentalhealth+research study', 'mentalhealth+new treatment', 'mentalhealth+volunteer', 'science+clinical trial', 'science+research study', 'science+new treatment', 'science+volunteer', 'cancer+clinical trial', 'cancer+research study', 'cancer+new treatment', 'cancer+volunteer'])

In [30]:
posts_li = data['clinicalresearch+clinical trial']['posts']

In [82]:
res = []
for i in trange(len(posts_li)):
    # print(posts_li[i]['title'])
    comments_li = posts_li[i]['comments']
    for j in trange(len(comments_li)):
        # print(comments_li[j]['body'])
        content = comments_li[j]['content']
        word_count = len(content.split())
        if word_count > 80:
            content2 = pipe_sum("In 50 words, Can you summarize the following text\n " + content)[0]['summary_text']
            # print(content2)
        else:
            content2 = content
            
        sa = pipe(content2)[0]
        # print(sa)
        # {'label': 'POS', 'score': 0.9905712604522705}
        label = sa['label']
        score = sa['score']
        res.append([content2, label, score])
        pass

  0%|          | 0/10 [00:00<?, ?it/s]Your max_length is set to 142, but your input_length is only 140. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=70)


In [31]:
content = comments_li[0]['content']

In [1]:
# Use a pipeline as a high-level helper

pipe = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis")


/opt/miniconda3/envs/sentiment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
result = pipe("I like this movie")
print(result)

[{'label': 'POS', 'score': 0.9905712604522705}]
